# Init: Load Libraries and Functions

In [ ]:
import pandas as pd
import spacy
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
import string
from textblob import TextBlob  
import os
import re

np.random.seed(67)

In [16]:
# Function below uses pre-built model for sentiment classification from TextBlob
# https://textblob.readthedocs.io/en/dev/quickstart.html

def polarity_scorer(input_text):
    """This function operates on a column in a data frame using apply().
    Takes a column as an input and returns a tuple of the polarity score and subjectivity score
    use .tolist() to split into separate columns, like here: https://stackoverflow.com/questions/29550414/how-to-split-column-of-tuples-in-pandas-dataframe"""
    text = TextBlob(input_text)
    polarity_score = text.sentiment.polarity
    subjectivity_score = text.sentiment.subjectivity
    
    return polarity_score, subjectivity_score

In [14]:
nlp = spacy.load('en_core_web_lg')
punctuations = string.punctuation #this is a python module which contains all the punctuations characters in English (and probably other languages too)
stopwords = list(STOP_WORDS)

def spacy_tokenizer(input_text):
    """removes stop words and punctuation from a document, converts all tokens to lower case
    and combines all tokens into one string.
    used in this example it appends a new column to a dataframe through apply()"""
    processed_text = re.sub(r"http\S+", '', input_text) # remove URLS, https://stackoverflow.com/questions/24399820/expression-to-remove-url-links-from-twitter-tweet
    mytokens = nlp(processed_text)
    mytokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [word for word in mytokens if word not in stopwords and word not in punctuations ]
    mytokens = " ".join([i for i in mytokens])
    return mytokens

In [15]:
# removing stopwords, it's this easy: https://medium.com/@makcedward/nlp-pipeline-stop-words-part-5-d6770df8a936

def token_parser(input_text):
    """This function creates a table with the text token and parts of a speech for a piece of text
    Builds one to many relationships by expanding all the tokens within a single document"""
    table = pd.DataFrame()
    doc = nlp(input_text)
    text = [token.text for token in doc if not token.is_stop]
    pos = [token.pos_ for token in doc if not token.is_stop]
    table['text'] = text
    table['pos'] = pos
    
    return table

# Import Data and process personalized subject lines

In [17]:
data = pd.read_csv('rep_email_detail incremental pull after 8.23.csv')

In [18]:
data.shape

(559, 19)

In [19]:
data.head()

,send_dt,send_time,send_number,campaign,email_name,category,subject,emails_sent,emails_delivered,undeliverable,total_clicks,unique_clicks,unique_opens,unsubscribes,unique_complaints,total_complaints,gifts,revenue,Audience
0,2022-08-26,14:49:44,1,Unknown,ML Insights Survey Monthly 2022 August - Reminder,Unknown,Insight Panel: Our team answers your questions,468,468,0,37,33,267,1,0,0,0,0.0,Other
1,2022-08-26,20:06:50,1,Monkeypox,2022-08-25-Monkeypox-0-12-Month-Donors,Cultivation,"%%[ VAR @DefaultName, @Salutation Set @Default...",131132,130411,721,890,522,42350,72,22,32,0,0.0,First Levels
2,2022-08-26,20:06:53,1,Monkeypox,2022-08-25-Monkeypox-Non-Donors,Cultivation,"%%[ VAR @DefaultName, @Salutation Set @Default...",224396,223572,824,647,478,62768,145,38,60,0,0.0,Non Donors
3,2022-08-26,20:07:59,1,Monkeypox,2022-08-25-Monkeypox-Field-Partner,Cultivation,"%%[ VAR @DefaultName, @Salutation Set @Default...",76001,75790,211,482,375,30205,20,6,8,0,0.0,Field Partners
4,2022-08-26,20:07:41,1,Monkeypox,2022-08-25-Monkeypox-Partner,Cultivation,"%%[ VAR @DefaultName, @Salutation Set @Default...",18226,18182,44,103,79,7347,5,4,6,0,0.0,Partners


In [20]:
data.subject.nunique()

149

In [21]:
data.rename({'subject':'text'}, axis=1, inplace=True)

In [43]:
data.text = [re.sub('[%%](.*)[%%]', '', text) for text in data.text]

In [44]:
data.head()

,send_dt,send_time,send_number,campaign,email_name,category,text,emails_sent,emails_delivered,undeliverable,total_clicks,unique_clicks,unique_opens,unsubscribes,unique_complaints,total_complaints,gifts,revenue,Audience
0,2022-08-26,14:49:44,1,Unknown,ML Insights Survey Monthly 2022 August - Reminder,Unknown,Insight Panel: Our team answers your questions,468,468,0,37,33,267,1,0,0,0,0.0,Other
1,2022-08-26,20:06:50,1,Monkeypox,2022-08-25-Monkeypox-0-12-Month-Donors,Cultivation,Here are th,131132,130411,721,890,522,42350,72,22,32,0,0.0,First Levels
2,2022-08-26,20:06:53,1,Monkeypox,2022-08-25-Monkeypox-Non-Donors,Cultivation,Here are th,224396,223572,824,647,478,62768,145,38,60,0,0.0,Non Donors
3,2022-08-26,20:07:59,1,Monkeypox,2022-08-25-Monkeypox-Field-Partner,Cultivation,Here are th,76001,75790,211,482,375,30205,20,6,8,0,0.0,Field Partners
4,2022-08-26,20:07:41,1,Monkeypox,2022-08-25-Monkeypox-Partner,Cultivation,Here are th,18226,18182,44,103,79,7347,5,4,6,0,0.0,Partners


In [46]:
data['send_group'] = data.email_name.str[:16]
data['Open_Rate_nw'] = data.unique_opens / data.emails_sent
data['Click_Rate_nw'] = data.unique_clicks / data.emails_sent
data['Donation_Rate_nw'] = data.gifts / data.emails_sent
data['revenue_1k_new'] = data.revenue / (data.emails_sent/1000)

In [47]:
data.send_dt = pd.to_datetime(data.send_dt)

In [48]:
data['month'] = pd.DatetimeIndex(data.send_dt).month

In [52]:
data.head(25)

,send_dt,send_time,send_number,campaign,email_name,category,text,emails_sent,emails_delivered,undeliverable,...,total_complaints,gifts,revenue,Audience,send_group,Open_Rate_nw,Click_Rate_nw,Donation_Rate_nw,revenue_1k_new,month
0,2022-08-26,14:49:44,1,Unknown,ML Insights Survey Monthly 2022 August - Reminder,Unknown,Insight Panel: Our team answers your questions,468,468,0,...,0,0,0.0,Other,ML Insights Surv,0.570513,0.070513,0.000000,0.000000,8
1,2022-08-26,20:06:50,1,Monkeypox,2022-08-25-Monkeypox-0-12-Month-Donors,Cultivation,Here are th,131132,130411,721,...,32,0,0.0,First Levels,2022-08-25-Monke,0.322957,0.003981,0.000000,0.000000,8
2,2022-08-26,20:06:53,1,Monkeypox,2022-08-25-Monkeypox-Non-Donors,Cultivation,Here are th,224396,223572,824,...,60,0,0.0,Non Donors,2022-08-25-Monke,0.279720,0.002130,0.000000,0.000000,8
3,2022-08-26,20:07:59,1,Monkeypox,2022-08-25-Monkeypox-Field-Partner,Cultivation,Here are th,76001,75790,211,...,8,0,0.0,Field Partners,2022-08-25-Monke,0.397429,0.004934,0.000000,0.000000,8
4,2022-08-26,20:07:41,1,Monkeypox,2022-08-25-Monkeypox-Partner,Cultivation,Here are th,18226,18182,44,...,6,0,0.0,Partners,2022-08-25-Monke,0.403105,0.004334,0.000000,0.000000,8
5,2022-08-26,20:06:53,1,Monkeypox,2022-08-25-Monkeypox-13-37-Month-Donors,Cultivation,Here are th,135921,135637,284,...,37,0,0.0,Lapsed Donors,2022-08-25-Monke,0.359621,0.001442,0.000000,0.000000,8
6,2022-08-29,17:22:00,1,Haiti,2022-08-29-Haiti-E1-13-37-Month-Donors,Sustainer Appeals,Haiti: Bringing health care to people trapped ...,130425,130205,220,...,22,3,182.0,Lapsed Donors,2022-08-29-Haiti,0.338593,0.001012,0.000023,1.395438,8
7,2022-08-29,17:22:05,1,Haiti,2022-08-29-Haiti-E1-Field-Partner,Cultivation,Haiti: Bringing health care to people trapped ...,70331,70157,174,...,9,15,752.0,Field Partners,2022-08-29-Haiti,0.373448,0.003768,0.000213,10.692298,8
8,2022-08-29,17:22:00,1,Haiti,2022-08-29-Haiti-E1-Non-Donors,Sustainer Appeals,Haiti: Bringing health care to people trapped ...,220245,219563,682,...,44,3,80.0,Non Donors,2022-08-29-Haiti,0.264592,0.001053,0.000014,0.363232,8
9,2022-08-29,17:21:55,1,Haiti,2022-08-29-Haiti-E1-0-12-Month-Donors,Sustainer Appeals,Haiti: Bringing health care to people trapped ...,125817,125252,565,...,32,31,2101.0,First Levels,2022-08-29-Haiti,0.293013,0.002218,0.000246,16.698856,8


# Processing new cols with NLP and Spacy

In [53]:
# calls functions through apply(), returns a tuple then splits the results into 2 columns
data.text = data.text.astype('str') 

data[['polarity_score', 'subjectivity_score']] = pd.DataFrame(data.text.apply(polarity_scorer).tolist(), index = data.index)

data.head()

,send_dt,send_time,send_number,campaign,email_name,category,text,emails_sent,emails_delivered,undeliverable,...,revenue,Audience,send_group,Open_Rate_nw,Click_Rate_nw,Donation_Rate_nw,revenue_1k_new,month,polarity_score,subjectivity_score
0,2022-08-26,14:49:44,1,Unknown,ML Insights Survey Monthly 2022 August - Reminder,Unknown,Insight Panel: Our team answers your questions,468,468,0,...,0.0,Other,ML Insights Surv,0.570513,0.070513,0.0,0.0,8,0.0,0.0
1,2022-08-26,20:06:50,1,Monkeypox,2022-08-25-Monkeypox-0-12-Month-Donors,Cultivation,Here are th,131132,130411,721,...,0.0,First Levels,2022-08-25-Monke,0.322957,0.003981,0.0,0.0,8,0.0,0.0
2,2022-08-26,20:06:53,1,Monkeypox,2022-08-25-Monkeypox-Non-Donors,Cultivation,Here are th,224396,223572,824,...,0.0,Non Donors,2022-08-25-Monke,0.279720,0.002130,0.0,0.0,8,0.0,0.0
3,2022-08-26,20:07:59,1,Monkeypox,2022-08-25-Monkeypox-Field-Partner,Cultivation,Here are th,76001,75790,211,...,0.0,Field Partners,2022-08-25-Monke,0.397429,0.004934,0.0,0.0,8,0.0,0.0
4,2022-08-26,20:07:41,1,Monkeypox,2022-08-25-Monkeypox-Partner,Cultivation,Here are th,18226,18182,44,...,0.0,Partners,2022-08-25-Monke,0.403105,0.004334,0.0,0.0,8,0.0,0.0


In [56]:
data['processed_text'] = data.text.apply(spacy_tokenizer)

In [57]:
data.head()

,send_dt,send_time,send_number,campaign,email_name,category,text,emails_sent,emails_delivered,undeliverable,...,Audience,send_group,Open_Rate_nw,Click_Rate_nw,Donation_Rate_nw,revenue_1k_new,month,polarity_score,subjectivity_score,processed_text
0,2022-08-26,14:49:44,1,Unknown,ML Insights Survey Monthly 2022 August - Reminder,Unknown,Insight Panel: Our team answers your questions,468,468,0,...,Other,ML Insights Surv,0.570513,0.070513,0.0,0.0,8,0.0,0.0,insight panel team answer question
1,2022-08-26,20:06:50,1,Monkeypox,2022-08-25-Monkeypox-0-12-Month-Donors,Cultivation,Here are th,131132,130411,721,...,First Levels,2022-08-25-Monke,0.322957,0.003981,0.0,0.0,8,0.0,0.0,th
2,2022-08-26,20:06:53,1,Monkeypox,2022-08-25-Monkeypox-Non-Donors,Cultivation,Here are th,224396,223572,824,...,Non Donors,2022-08-25-Monke,0.279720,0.002130,0.0,0.0,8,0.0,0.0,th
3,2022-08-26,20:07:59,1,Monkeypox,2022-08-25-Monkeypox-Field-Partner,Cultivation,Here are th,76001,75790,211,...,Field Partners,2022-08-25-Monke,0.397429,0.004934,0.0,0.0,8,0.0,0.0,th
4,2022-08-26,20:07:41,1,Monkeypox,2022-08-25-Monkeypox-Partner,Cultivation,Here are th,18226,18182,44,...,Partners,2022-08-25-Monke,0.403105,0.004334,0.0,0.0,8,0.0,0.0,th


In [58]:
data.to_csv('q4 2022 data raw.csv', index = False)

# Ingest file after results are scored with deeplearning model

In [13]:
emotion_scored_results = pd.read_csv('Q4 2022 emotion outputs full v1.csv')

In [15]:
emotion_scored_results['emo value_original'] = emotion_scored_results.max(axis='columns')

In [17]:
emotion_scored_results['emo_class_original'] = emotion_scored_results.idxmax(axis='columns')

In [18]:
emotion_scored_results.to_csv('Q4 2022 emotion outputs full v1_update.csv')

# Appendix

In [23]:
data.loc[1,'text']

"%%[ VAR @DefaultName, @Salutation Set @DefaultName = 'Friend' IF LENGTH([FirstName]) > 1 THEN Set @Salutation = Concat(ProperCase([FirstName])) ELSE set @Salutation = @DefaultName ENDIF)]%%Here are th"

In [24]:
test = data.loc[1,'text']
test

"%%[ VAR @DefaultName, @Salutation Set @DefaultName = 'Friend' IF LENGTH([FirstName]) > 1 THEN Set @Salutation = Concat(ProperCase([FirstName])) ELSE set @Salutation = @DefaultName ENDIF)]%%Here are th"

In [29]:
re.sub('(?<=%%)(.*)(?=%%)', '', test)

'%%%%Here are th'

In [32]:
# https://learnbyexample.github.io/python-regex-cheatsheet/
# https://stackoverflow.com/questions/1454913/
# regular-expression-to-find-a-string-included-between-two-characters-while-exclud/1454936#1454936

re.sub('[%%](.*)[%%]', '', test)

'Here are th'

In [33]:
data.text = re.sub('[%%](.*)[%%]', '', data.text)

TypeError: expected string or bytes-like object

In [41]:
subject_cleaned = [re.sub('[%%](.*)[%%]', '', text) for text in data.text]

In [42]:
subject_cleaned

['Insight Panel: Our team answers your questions',
 'Here are th',
 'Here are th',
 'Here are th',
 'Here are th',
 'Here are th',
 'Haiti: Bringing health care to people trapped by violence',
 'Haiti: Bringing health care to people trapped by violence',
 'Haiti: Bringing health care to people trapped by violence',
 'Haiti: Bringing health care to people trapped by violence',
 'Haiti: Bringing health care to people trapped by violence',
 ', Your IRA can save lives',
 ', Your IRA can save lives',
 'Responding to Afghanistan?s spiraling crisis',
 'Responding to Afghanistan?s spiraling crisis',
 'Responding to Afghanistan?s spiraling crisis',
 'Please Renew Your Support of Doctors Without Borders',
 'Please Renew Your Support of Doctors Without Borders',
 'Join our Virtual Mapathon on September 8',
 'EMERGENCY ALERT: One third of Pakistan underwater',
 'EMERGENCY ALERT: One third of Pakistan underwater',
 'EMERGENCY ALERT: One third of Pakistan underwater',
 'EMERGENCY ALERT: One third of